In [7]:
#!pip install folium
#!pip install pyproj

In [8]:
import pandas as pd
import folium
import math

from itertools import combinations
from pyproj import Proj, transform
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

### 데이터 다운 및 전처리

In [10]:
Busan = pd.read_excel('부산시예상좌표.xlsx')
Busan=Busan[Busan['시군구']=='기장군']
Busan = Busan[['대구분명칭', '위도', '경도']]
Busan.columns = ['이름', '위도', '경도']
Busan.head()

,이름,위도,경도
0,장안읍 임랑리 아름마을,35.316338,129.259625
1,한신아파트 앞 주변 이면도로,35.248296,129.216662
2,이천마을 공영주차장,35.267603,129.236131
3,광장 11호,35.330460,129.191100
4,두명소공원,35.361791,129.146673


In [11]:
market = pd.read_excel('부산_편의점.xlsx')
market.head()

,이름,위도,경도
0,이마트24 기장공수점,35.183735,129.210503
1,이마트24 해동용궁사점,35.189304,129.221113
2,이마트24 송정휴먼시아점,35.190819,129.204996
3,세븐일레븐 동부산아울렛3호점,35.191556,129.212524
4,이마트24 기장스타테라스점,35.192019,129.217766


### 위,경도 좌표변환

In [12]:
Busan.reset_index(drop=True, inplace=True)

In [13]:
proj_UTMK = Proj(init='epsg:5178')
proj_WGS84 = Proj(init = 'epsg:4326')

for i in  range(len(Busan)) :
    x1, y1 = transform(proj_WGS84, proj_UTMK, Busan['경도'][i], Busan['위도'][i])
    Busan['경도'][i], Busan['위도'][i] = x1, y1
    
for i in  range(len(market)) :
    x1, y1 = transform(proj_WGS84, proj_UTMK, market['경도'][i], market['위도'][i])
    market['경도'][i], market['위도'][i] = x1, y1  

### F와 L사이의 최단거리 행열 만들기

$F_i$ - Busan, 부산광역시 기장군 공영시설 좌표, m = 85

$L_j$ - market, 부산광역시 편의점 좌표, n = 152

$c_{ij}$ - $F_i$와 $L_j$의 최단거리

In [14]:
F_list = []
L_list = []
for i in range(len(Busan)) :
    name = 'F_{}'.format(i)
    F_list.append(name)
for i in range(len(market)) :
    name = 'L_{}'.format(i)
    L_list.append(name)

In [15]:
length_df = pd.DataFrame(columns = F_list, index = L_list)
for i in range(len(length_df)) :
    for j, col in enumerate(length_df.columns) :
        square_sum = ((Busan['위도'][j] - market['위도'][i]) ** 2) + ((Busan['경도'][j] - market['경도'][i]) ** 2)
        distance = math.sqrt(square_sum)
        length_df[col][i] = distance

###  P-Median

In [16]:
char_list = []
for p in range(3, 11) :
    col_sum = list(length_df.sum(axis = 0))
    col_sum_sort = sorted(col_sum)
    min_value = col_sum_sort[:3]
    
    min_index = []
    for value in min_value :
        min_index.append(col_sum.index(value))
    
    col_sum_tuple = []
    for i in range(len(col_sum)) :
        tup = (i, col_sum[i])
        col_sum_tuple.append(tup)
    
    col_sum_tuple.sort(key = lambda x: x[1])
    col_sum_tuple = col_sum_tuple[:2*p]
    
    p_list_set = [col_sum_tuple[j:j+p] for j in range(p)]
    
    min_sum_list = []
    for k in range(len(p_list_set)) :
        tup_check = []
        for tt in p_list_set[k] :
            tup_check.append(f'F_{tt[0]}')
        check_df = length_df[tup_check]
        check_df['min'] = 0
        for l in range(len(check_df)) :
            check_df['min'][l] = min(check_df.iloc[l][:-1])
        min_sum_value = sum(check_df['min'])
        min_sum_list.append(min_sum_value)
    
    final_index = min_sum_list.index(min(min_sum_list))
    final_set = p_list_set[final_index]
    
    final_set.sort(key = lambda x : x[0])
    
    final_idx = [tp[0] for tp in final_set]
    final_Busan = Busan.iloc[final_idx, :]
    
    final_Busan.reset_index(drop = True, inplace = True)
    char_list.append(char for char in final_Busan['이름'])

In [17]:
final_kk = []
for ch in char_list :
    final_kk += ch

In [18]:
pd.Series(final_kk).value_counts()

한신아파트 앞 주변 이면도로        6
일광해수욕장노상공영주차장          6
일광이천생태공원               6
이천마을 공영주차장             5
일광해수욕장 주변              5
동부근린공원 인근 노외 공영 주차장    4
기장시장공영주차장              4
일광면사무소주변               3
동산그린타워 주변 이면도로         3
동부근린공영주차장              3
기장시장 공영주차장             3
기장 제1공영주차장             2
동부근린공원                 1
교리근린공원                 1
dtype: int64

In [40]:
check_index = pd.Series(final_kk).value_counts().index[:3]

In [41]:
Busan_index = []
for check in check_index :
    for i in range(len(Busan)) :
        if Busan['이름'][i] == check :
            Busan_index.append(i)
            break

In [42]:
Busan_index

[1, 21, 66]

In [43]:
Busan_sub = Busan.iloc[Busan_index, :]
Busan_sub.reset_index(drop = True, inplace = True)
Busan_sub

,이름,위도,경도
0,한신아파트 앞 주변 이면도로,1.695809e+06,1.156376e+06
1,일광해수욕장노상공영주차장,1.697304e+06,1.157844e+06
2,일광이천생태공원,1.698402e+06,1.157996e+06


### 시장데이터 및 최종공영시설 좌표 위경도 변환

In [44]:
Busan_sijang = pd.read_excel('부산시장.xlsx')
Busan_sijang.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [45]:
Busan_sijang = Busan_sijang[['시장명', '위도', '경도']]

In [46]:
Busan_sijang=Busan_sijang.iloc[:2,:]

In [47]:
for i in  tqdm(range(len(Busan_sijang))) :
    x1, y1 = transform(proj_UTMK, proj_WGS84, Busan_sijang['경도'][i], Busan_sijang['위도'][i])
    Busan_sijang['경도'][i], Busan_sijang['위도'][i] = x1, y1

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 10.36it/s]


In [48]:
for i in  tqdm(range(len(Busan_sub))) :
    x1, y1 = transform(proj_UTMK, proj_WGS84, Busan_sub['경도'][i], Busan_sub['위도'][i])
    Busan_sub['경도'][i], Busan_sub['위도'][i] = x1, y1

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 10.56it/s]


In [49]:
m = folium.Map(location = [35.18080954115435, 129.0750855720365], zoom_start = 10)

In [50]:
#m.save('map.html')
# m

In [51]:
for i in range(len(Busan_sijang)) :
    long = Busan_sijang['위도'][i]
    lat = Busan_sijang['경도'][i]
    name = Busan_sijang['시장명'][i]
    folium.Marker(location = [long, lat],
                  popup = name,
                 icon = folium.Icon(color = 'yellow',
                                    icon = 'star',
                                    size = 0.1)).add_to(m)

In [52]:
for i in range(len(Busan_sub)) :
    long = Busan_sub['위도'][i]
    lat = Busan_sub['경도'][i]
    name = Busan_sub['이름'][i]
    folium.Marker(location = [long, lat],
                  popup = name,
                 icon = folium.Icon(color = 'blue',
                                    icon = 'flag',
                                    size = 0.1)).add_to(m)

In [53]:
m